In [10]:
import shapely as sh
import matplotlib.pyplot as plt
%matplotlib qt
from phidl.quickplotter import quickplot2 as qp
import numpy as np
from geomdl import NURBS,knotvector
import utilities as util
from ipywidgets import interact,FloatSlider
def normal_ls_at_frac(ls,frac,tol=1e-3):
    if frac==0:
        pts=ls.interpolate([frac,frac+tol],normalized=True)
        x2,y2 = np.asarray(pts[0].xy)
        x1,y1 = np.asarray(pts[1].xy)
    else:
        pts=ls.interpolate([frac-tol,frac],normalized=True)
        x1,y1 = np.asarray(pts[0].xy)
        x2,y2 = np.asarray(pts[1].xy)
    dx,dy = x1-x2,y1-y2
    norm = np.array([dy,-dx])/np.sqrt(dx*dx+dy*dy)
    return norm
def wiggly_ring(r,rmod,n_wiggles):
    t = np.linspace(0,2*np.pi,1000)
    rad = r + rmod*np.cos(n_wiggles*t)
    x,y = rad*np.cos(t),rad*np.sin(t)
    return sh.LineString( np.vstack((x,y)).T)

def make_latch(ls1,ls2,frac1,frac2,mag1,mag2,ang1,ang2,res_latch=100):
    norm_out = normal_ls_at_frac(outer_ring,frac1)
    norm_in  = normal_ls_at_frac(inner_ring,frac2)

    p1 = ls1.interpolate(frac1,normalized=True)
    p4 = ls2.interpolate(frac2,normalized=True)
    p2 = sh.affinity.translate(p1, xoff=mag1*norm_out[0],yoff=mag1*norm_out[1])
    p2 = sh.affinity.rotate(p2,origin=p1,angle=-ang1)
    p3 = sh.affinity.translate(p4, xoff=-mag2*norm_in[0],yoff=-mag2*norm_in[1])
    p3 = sh.affinity.rotate(p3,origin=p4,angle=-ang2)
    ctrl_pts = sh.MultiPoint([p1,p2,p3,p4])

    curve=NURBS.Curve()
    curve.degree=3
    curve.ctrlpts=np.asarray([el.xy for el in ctrl_pts.geoms])
    curve.knotvector = knotvector.generate(curve.degree, len(curve.ctrlpts))
    curve.delta=1./res_latch
    latch=sh.LineString(np.asarray(curve.evalpts))
    return latch

In [11]:
outer_ring_rad = 1000
out_mod = 50
n_wiggles=16
inner_ring_rad = 350
in_mod = 20
n_latches = 8
outer_ring=wiggly_ring(outer_ring_rad,out_mod,n_wiggles)
inner_ring=wiggly_ring(inner_ring_rad,in_mod,n_wiggles)



In [12]:
#compute normal to linestring
ls1 = outer_ring
frac1=.11
mag1=200
ang1=10
ls2 = inner_ring
frac2=.36
mag2=300
ang2=0

def parametric_device(mag1,ang1):
    latches=[]
    conn_in = []
    conn_out = []
    for i in range(n_latches):
        shift = i/n_latches
        f1=(frac1+shift)%1
        f2=(frac2+shift)%1
        latch = make_latch(ls1,ls2,f1,f2,mag1,mag2,ang1,ang2)
        latches.append(latch)
        
        conn_out.append(outer_ring.interpolate(f1,normalized=True))
        conn_in.append(inner_ring.interpolate(f2,normalized=True))
    dev = outer_ring.buffer(20).union(inner_ring.buffer(10)).union(sh.unary_union(latches).buffer(10))
    cutout_out = sh.unary_union(conn_out).buffer(100) #cutout circles
    cutout_in = sh.unary_union(conn_in).buffer(50)
    
    rounded_conn_in = dev.intersection(cutout_in).buffer(35).buffer(-35)
    rounded_conn_out = dev.intersection(cutout_out).buffer(35).buffer(-35)
    
    dev=dev.union(rounded_conn_in).union(rounded_conn_out).buffer(5).buffer(-5)
    
    #qp(util.shapely_to_phidl(dev))
    #dev=dev#.buffer(35).buffer(-35)
    return dev

dev=parametric_device(1000,0)
qp(util.shapely_to_phidl(dev))

In [48]:
parameters=[]
for ang in np.linspace(0,60,20):
    for mag in np.linspace(0,1000,20):
        if mag <1000-720/60*ang: #be under the curve
            parameters.append([mag,ang])
parameters=np.asarray(parameters)
plt.scatter(parameters[:,0],parameters[:,1])

In [16]:
res=25

import meshio,pygmsh,gmsh
def remap(line,resolution_um=10):
    return sh.LineString(line.interpolate(np.linspace(0,1,int(line.length/resolution_um)),normalized=True))

def create_mesh(mesh, cell_type):
    cells = mesh.get_cells_type(cell_type)
    cell_data = mesh.get_cell_data("gmsh:physical", cell_type)
    out_mesh = meshio.Mesh(
        points=mesh.points[:,:2]/outer_ring_rad, cells={cell_type: cells}, cell_data={"markers": [cell_data]}
    )
    return out_mesh



for idx,(p1,p2) in enumerate(parameters):
    dev = parametric_device(p1,p2)
    with pygmsh.geo.Geometry() as geom:

        holes = [geom.add_polygon(np.asarray(remap(el,res).coords)[:-1,:]) for el in dev.interiors]
        perimeter = geom.add_polygon(np.asarray(remap(dev.exterior,res).coords)[:-1,:],holes=holes)


        geom.add_physical(perimeter.lines[:], "exterior")   # 3

        geom.add_physical(perimeter.surface, "domain")      # 5

        mesh = geom.generate_mesh(dim=2, verbose=True)


        gmsh.write("tmp.msh")
        gmsh.clear()
        mesh = meshio.read("tmp.msh")

        triangle_mesh = create_mesh(mesh, "triangle")
        line_mesh = create_mesh(mesh, "line")

        meshio.write("meshes/mesh_2d_"+str(idx).zfill(4)+".xdmf", triangle_mesh)
        meshio.write("meshes/mesh_1d_"+str(idx).zfill(4)+".xdmf", line_mesh)
    x,y= triangle_mesh.points.T
    triangles = triangle_mesh.cells[0].data
    plt.triplot(x,y,triangles=triangles)
    plt.savefig('meshes/'+str(idx).zfill(4)+'.png')
    plt.close()

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 10%] Meshing curve 3 (Line)
Info    : [ 10%] Meshing curve 4 (Line)
Info    : [ 10%] Meshing curve 5 (Line)
Info    : [ 10%] Meshing curve 6 (Line)
Info    : [ 10%] Meshing curve 7 (Line)
Info    : [ 10%] Meshing curve 8 (Line)
Info    : [ 10%] Meshing curve 9 (Line)
Info    : [ 10%] Meshing curve 10 (Line)
Info    : [ 10%] Meshing curve 11 (Line)
Info    : [ 10%] Meshing curve 12 (Line)
Info    : [ 10%] Meshing curve 13 (Line)
Info    : [ 10%] Meshing curve 14 (Line)
Info    : [ 10%] Meshing curve 15 (Line)
Info    : [ 10%] Meshing curve 16 (Line)
Info    : [ 10%] Meshing curve 17 (Line)
Info    : [ 10%] Meshing curve 18 (Line)
Info    : [ 10%] Meshing curve 19 (Line)
Info    : [ 10%] Meshing curve 20 (Line)
Info    : [ 10%] Meshing curve 21 (Line)
Info    : [ 10%] Meshing curve 22 (Line)
Info    : [ 10%] Meshing curve 23 (Line)
Info    : [ 10%] Meshing curve 24 (Line)
I

Info    : Done meshing 2D (Wall 0.135424s, CPU 0.135952s)
Info    : 6478 nodes 15507 elements
Info    : Writing 'karm_test.msh'...
Info    : Done writing 'karm_test.msh'
Info    : Clearing all models and views...
Info    : Done clearing all models and views



In [63]:
x,y=triangle_mesh.points.T
tri=triangle_mesh.cells[0].data
plt.triplot(x,y,triangles=tri)
bdy_idx=line_mesh.cells[0].data[:,0]

plt.scatter(x[bdy_idx],y[bdy_idx],color='r')

In [7]:
import os
for i in range(len(parameters)):
    string = 'python3.10 fenics_simulation.py '+str(i)
    os.system(string)

Building point search tree to accelerate distance queries.
Computed bounding box tree with 4483 nodes for 2242 points.
Newton iteration 0: r (abs) = 0.000e+00 (tol = 1.000e-10) r (rel) = -nan (tol = 1.000e-09)
Newton solver finished in 0 iterations and 0 linear solver iterations.
Newton iteration 0: r (abs) = 2.059e-09 (tol = 1.000e-10) r (rel) = 1.000e+00 (tol = 1.000e-09)
Newton iteration 1: r (abs) = 2.015e-02 (tol = 1.000e-10) r (rel) = 9.788e+06 (tol = 1.000e-09)
Newton iteration 2: r (abs) = 9.464e-06 (tol = 1.000e-10) r (rel) = 4.596e+03 (tol = 1.000e-09)
Newton iteration 3: r (abs) = 4.508e-03 (tol = 1.000e-10) r (rel) = 2.189e+06 (tol = 1.000e-09)
Newton iteration 4: r (abs) = 3.833e-06 (tol = 1.000e-10) r (rel) = 1.862e+03 (tol = 1.000e-09)
Newton iteration 5: r (abs) = 9.606e-04 (tol = 1.000e-10) r (rel) = 4.665e+05 (tol = 1.000e-09)
Newton iteration 6: r (abs) = 9.181e-07 (tol = 1.000e-10) r (rel) = 4.459e+02 (tol = 1.000e-09)
Newton iteration 7: r (abs) = 2.392e-05 (tol = 

Newton iteration 1: r (abs) = 2.149e-02 (tol = 1.000e-10) r (rel) = 1.044e+07 (tol = 1.000e-09)
Newton iteration 2: r (abs) = 3.937e-06 (tol = 1.000e-10) r (rel) = 1.912e+03 (tol = 1.000e-09)
Newton iteration 3: r (abs) = 1.012e-04 (tol = 1.000e-10) r (rel) = 4.915e+04 (tol = 1.000e-09)
Newton iteration 4: r (abs) = 1.268e-06 (tol = 1.000e-10) r (rel) = 6.159e+02 (tol = 1.000e-09)
Newton iteration 5: r (abs) = 4.450e-05 (tol = 1.000e-10) r (rel) = 2.161e+04 (tol = 1.000e-09)
Newton iteration 6: r (abs) = 5.859e-08 (tol = 1.000e-10) r (rel) = 2.845e+01 (tol = 1.000e-09)
Newton iteration 7: r (abs) = 3.275e-05 (tol = 1.000e-10) r (rel) = 1.590e+04 (tol = 1.000e-09)
Newton iteration 8: r (abs) = 5.159e-09 (tol = 1.000e-10) r (rel) = 2.505e+00 (tol = 1.000e-09)
Newton iteration 9: r (abs) = 3.705e-07 (tol = 1.000e-10) r (rel) = 1.799e+02 (tol = 1.000e-09)
Newton iteration 10: r (abs) = 1.644e-11 (tol = 1.000e-10) r (rel) = 7.986e-03 (tol = 1.000e-09)
Newton solver finished in 10 iterations

Newton iteration 4: r (abs) = 1.523e-09 (tol = 1.000e-10) r (rel) = 7.398e-01 (tol = 1.000e-09)
Newton iteration 5: r (abs) = 1.326e-12 (tol = 1.000e-10) r (rel) = 6.438e-04 (tol = 1.000e-09)
Newton solver finished in 5 iterations and 5 linear solver iterations.
Newton iteration 0: r (abs) = 2.059e-09 (tol = 1.000e-10) r (rel) = 1.000e+00 (tol = 1.000e-09)
Newton iteration 1: r (abs) = 1.716e-02 (tol = 1.000e-10) r (rel) = 8.334e+06 (tol = 1.000e-09)
Newton iteration 2: r (abs) = 1.062e-06 (tol = 1.000e-10) r (rel) = 5.157e+02 (tol = 1.000e-09)
Newton iteration 3: r (abs) = 6.468e-08 (tol = 1.000e-10) r (rel) = 3.141e+01 (tol = 1.000e-09)
Newton iteration 4: r (abs) = 1.485e-09 (tol = 1.000e-10) r (rel) = 7.210e-01 (tol = 1.000e-09)
Newton iteration 5: r (abs) = 1.499e-12 (tol = 1.000e-10) r (rel) = 7.280e-04 (tol = 1.000e-09)
Newton solver finished in 5 iterations and 5 linear solver iterations.
Newton iteration 0: r (abs) = 2.059e-09 (tol = 1.000e-10) r (rel) = 1.000e+00 (tol = 1.000

Newton iteration 2: r (abs) = 8.731e-07 (tol = 1.000e-10) r (rel) = 4.240e+02 (tol = 1.000e-09)
Newton iteration 3: r (abs) = 7.129e-08 (tol = 1.000e-10) r (rel) = 3.462e+01 (tol = 1.000e-09)
Newton iteration 4: r (abs) = 1.104e-09 (tol = 1.000e-10) r (rel) = 5.361e-01 (tol = 1.000e-09)
Newton iteration 5: r (abs) = 2.188e-12 (tol = 1.000e-10) r (rel) = 1.063e-03 (tol = 1.000e-09)
Newton solver finished in 5 iterations and 5 linear solver iterations.
Newton iteration 0: r (abs) = 2.059e-09 (tol = 1.000e-10) r (rel) = 1.000e+00 (tol = 1.000e-09)
Newton iteration 1: r (abs) = 1.613e-02 (tol = 1.000e-10) r (rel) = 7.835e+06 (tol = 1.000e-09)
Newton iteration 2: r (abs) = 8.585e-07 (tol = 1.000e-10) r (rel) = 4.170e+02 (tol = 1.000e-09)
Newton iteration 3: r (abs) = 7.214e-08 (tol = 1.000e-10) r (rel) = 3.504e+01 (tol = 1.000e-09)
Newton iteration 4: r (abs) = 1.076e-09 (tol = 1.000e-10) r (rel) = 5.225e-01 (tol = 1.000e-09)
Newton iteration 5: r (abs) = 2.108e-12 (tol = 1.000e-10) r (rel)

Newton iteration 1: r (abs) = 1.530e-02 (tol = 1.000e-10) r (rel) = 7.432e+06 (tol = 1.000e-09)
Newton iteration 2: r (abs) = 6.965e-07 (tol = 1.000e-10) r (rel) = 3.383e+02 (tol = 1.000e-09)
Newton iteration 3: r (abs) = 8.515e-08 (tol = 1.000e-10) r (rel) = 4.135e+01 (tol = 1.000e-09)
Newton iteration 4: r (abs) = 8.244e-10 (tol = 1.000e-10) r (rel) = 4.004e-01 (tol = 1.000e-09)
Newton iteration 5: r (abs) = 3.115e-12 (tol = 1.000e-10) r (rel) = 1.513e-03 (tol = 1.000e-09)
Newton solver finished in 5 iterations and 5 linear solver iterations.
Newton iteration 0: r (abs) = 2.059e-09 (tol = 1.000e-10) r (rel) = 1.000e+00 (tol = 1.000e-09)
Newton iteration 1: r (abs) = 1.523e-02 (tol = 1.000e-10) r (rel) = 7.398e+06 (tol = 1.000e-09)
Newton iteration 2: r (abs) = 6.839e-07 (tol = 1.000e-10) r (rel) = 3.322e+02 (tol = 1.000e-09)
Newton iteration 3: r (abs) = 8.657e-08 (tol = 1.000e-10) r (rel) = 4.204e+01 (tol = 1.000e-09)
Newton iteration 4: r (abs) = 8.105e-10 (tol = 1.000e-10) r (rel)

...

Newton iteration 1: r (abs) = 1.177e-02 (tol = 1.000e-10) r (rel) = 5.736e+06 (tol = 1.000e-09)
Newton iteration 2: r (abs) = 3.295e-07 (tol = 1.000e-10) r (rel) = 1.606e+02 (tol = 1.000e-09)
Newton iteration 3: r (abs) = 1.765e-07 (tol = 1.000e-10) r (rel) = 8.601e+01 (tol = 1.000e-09)
Newton iteration 4: r (abs) = 3.881e-10 (tol = 1.000e-10) r (rel) = 1.891e-01 (tol = 1.000e-09)
Newton iteration 5: r (abs) = 6.172e-12 (tol = 1.000e-10) r (rel) = 3.007e-03 (tol = 1.000e-09)
Newton solver finished in 5 iterations and 5 linear solver iterations.
Newton iteration 0: r (abs) = 2.052e-09 (tol = 1.000e-10) r (rel) = 1.000e+00 (tol = 1.000e-09)
Newton iteration 1: r (abs) = 1.164e-02 (tol = 1.000e-10) r (rel) = 5.674e+06 (tol = 1.000e-09)
Newton iteration 2: r (abs) = 3.259e-07 (tol = 1.000e-10) r (rel) = 1.588e+02 (tol = 1.000e-09)
Newton iteration 3: r (abs) = 1.801e-07 (tol = 1.000e-10) r (rel) = 8.776e+01 (tol = 1.000e-09)
Newton iteration 4: r (abs) = 3.877e-10 (tol = 1.000e-10) r (rel)

Newton iteration 1: r (abs) = 1.152e-02 (tol = 1.000e-10) r (rel) = 5.612e+06 (tol = 1.000e-09)
Newton iteration 2: r (abs) = 3.231e-07 (tol = 1.000e-10) r (rel) = 1.574e+02 (tol = 1.000e-09)
Newton iteration 3: r (abs) = 1.845e-07 (tol = 1.000e-10) r (rel) = 8.990e+01 (tol = 1.000e-09)
Newton iteration 4: r (abs) = 3.922e-10 (tol = 1.000e-10) r (rel) = 1.911e-01 (tol = 1.000e-09)
Newton iteration 5: r (abs) = 6.484e-12 (tol = 1.000e-10) r (rel) = 3.159e-03 (tol = 1.000e-09)
Newton solver finished in 5 iterations and 5 linear solver iterations.
Newton iteration 0: r (abs) = 2.052e-09 (tol = 1.000e-10) r (rel) = 1.000e+00 (tol = 1.000e-09)
Newton iteration 1: r (abs) = 1.139e-02 (tol = 1.000e-10) r (rel) = 5.550e+06 (tol = 1.000e-09)
Newton iteration 2: r (abs) = 3.211e-07 (tol = 1.000e-10) r (rel) = 1.565e+02 (tol = 1.000e-09)
Newton iteration 3: r (abs) = 1.893e-07 (tol = 1.000e-10) r (rel) = 9.226e+01 (tol = 1.000e-09)
Newton iteration 4: r (abs) = 4.018e-10 (tol = 1.000e-10) r (rel)

Newton iteration 6: r (abs) = 6.482e-07 (tol = 1.000e-10) r (rel) = 3.157e+02 (tol = 1.000e-09)
Newton iteration 7: r (abs) = 2.677e-05 (tol = 1.000e-10) r (rel) = 1.304e+04 (tol = 1.000e-09)
Newton iteration 8: r (abs) = 7.475e-08 (tol = 1.000e-10) r (rel) = 3.641e+01 (tol = 1.000e-09)
Newton iteration 9: r (abs) = 7.012e-08 (tol = 1.000e-10) r (rel) = 3.415e+01 (tol = 1.000e-09)
Newton iteration 10: r (abs) = 5.459e-13 (tol = 1.000e-10) r (rel) = 2.659e-04 (tol = 1.000e-09)
Newton solver finished in 10 iterations and 10 linear solver iterations.
Newton iteration 0: r (abs) = 2.053e-09 (tol = 1.000e-10) r (rel) = 1.000e+00 (tol = 1.000e-09)
Newton iteration 1: r (abs) = 1.836e-02 (tol = 1.000e-10) r (rel) = 8.943e+06 (tol = 1.000e-09)
Newton iteration 2: r (abs) = 1.358e-06 (tol = 1.000e-10) r (rel) = 6.617e+02 (tol = 1.000e-09)
Newton iteration 3: r (abs) = 2.122e-04 (tol = 1.000e-10) r (rel) = 1.033e+05 (tol = 1.000e-09)
Newton iteration 4: r (abs) = 2.066e-07 (tol = 1.000e-10) r (r

Newton iteration 3: r (abs) = 3.262e-06 (tol = 1.000e-10) r (rel) = 1.589e+03 (tol = 1.000e-09)
Newton iteration 4: r (abs) = 8.831e-08 (tol = 1.000e-10) r (rel) = 4.301e+01 (tol = 1.000e-09)
Newton iteration 5: r (abs) = 4.334e-08 (tol = 1.000e-10) r (rel) = 2.111e+01 (tol = 1.000e-09)
Newton iteration 6: r (abs) = 4.915e-11 (tol = 1.000e-10) r (rel) = 2.394e-02 (tol = 1.000e-09)
Newton solver finished in 6 iterations and 6 linear solver iterations.
Newton iteration 0: r (abs) = 2.054e-09 (tol = 1.000e-10) r (rel) = 1.000e+00 (tol = 1.000e-09)
Newton iteration 1: r (abs) = 2.040e-02 (tol = 1.000e-10) r (rel) = 9.931e+06 (tol = 1.000e-09)
Newton iteration 2: r (abs) = 1.559e-06 (tol = 1.000e-10) r (rel) = 7.594e+02 (tol = 1.000e-09)
Newton iteration 3: r (abs) = 1.007e-06 (tol = 1.000e-10) r (rel) = 4.906e+02 (tol = 1.000e-09)
Newton iteration 4: r (abs) = 3.090e-08 (tol = 1.000e-10) r (rel) = 1.505e+01 (tol = 1.000e-09)
Newton iteration 5: r (abs) = 1.030e-09 (tol = 1.000e-10) r (rel)

Newton iteration 1: r (abs) = 1.870e-02 (tol = 1.000e-10) r (rel) = 9.106e+06 (tol = 1.000e-09)
Newton iteration 2: r (abs) = 1.235e-06 (tol = 1.000e-10) r (rel) = 6.013e+02 (tol = 1.000e-09)
Newton iteration 3: r (abs) = 7.182e-08 (tol = 1.000e-10) r (rel) = 3.498e+01 (tol = 1.000e-09)
Newton iteration 4: r (abs) = 2.540e-09 (tol = 1.000e-10) r (rel) = 1.237e+00 (tol = 1.000e-09)
Newton iteration 5: r (abs) = 1.661e-12 (tol = 1.000e-10) r (rel) = 8.091e-04 (tol = 1.000e-09)
Newton solver finished in 5 iterations and 5 linear solver iterations.
Newton iteration 0: r (abs) = 2.053e-09 (tol = 1.000e-10) r (rel) = 1.000e+00 (tol = 1.000e-09)
Newton iteration 1: r (abs) = 1.859e-02 (tol = 1.000e-10) r (rel) = 9.057e+06 (tol = 1.000e-09)
Newton iteration 2: r (abs) = 1.213e-06 (tol = 1.000e-10) r (rel) = 5.910e+02 (tol = 1.000e-09)
Newton iteration 3: r (abs) = 7.221e-08 (tol = 1.000e-10) r (rel) = 3.518e+01 (tol = 1.000e-09)
Newton iteration 4: r (abs) = 2.484e-09 (tol = 1.000e-10) r (rel)

In [139]:
displacements=[]
strains=[]
from scipy.stats import iqr
for i in range(1,len(parameters)):
    try:
        m=meshio.read('output/'+str(i).zfill(4)+'_configuration000099.vtu')
        x,y,_=m.points.T
        tri=m.cells[0].data
        max_displacement = np.min(m.point_data['phi'][:,-1])
        displacements.append(max_displacement)
        
        ms=meshio.read('output/'+str(i).zfill(4)+'_strain000099.vtu')
        strain=np.sqrt(np.sum(ms.point_data['Strain']**2,axis=1))
        
        strains.append(np.min(strain)+ iqr(strain,rng=(0,99)))
    except:
        displacements.append(0)
        strains.append(0)
        continue

In [142]:
import matplotlib.cm as cm
plt.scatter(parameters[1:,0],parameters[1:,1],c=cm.viridis(displacements/np.min(displacements)))
plt.colorbar()

In [141]:
plt.scatter(parameters[1:,0],parameters[1:,1],c=cm.viridis(strains/np.max(strains)))


In [181]:
def save_for_blender(infile_idx,outfile):
    m=meshio.read('output/'+str(infile_idx).zfill(4)+'_configuration000099.vtu')
    m.points+=m.point_data['phi']
    
    ms=meshio.read('output/'+str(infile_idx).zfill(4)+'_strain000099.vtu')
    strain=np.sqrt(np.sum(ms.point_data['Strain']**2,axis=1))

    meshio.write('meshes/'+outfile+'.stl',m)
    order_of_sight=m.cells[0].data.flatten()
    reorder_idx = [order_of_sight[0]]
    for el in order_of_sight[1:]:
        if el not in reorder_idx:
            reorder_idx.append(el)
    strain=strain[reorder_idx] #blender re-orders vertex indices from the stl file as [0,1,2],[3,4,5],etc..
    
    np.savetxt('meshes/strain'+outfile+'.txt',strain/np.max(strain))

save_for_blender(87,'good')
save_for_blender(32,'bad')

In [199]:
infile_idx=87
m=meshio.read('output/'+str(infile_idx).zfill(4)+'_configuration000099.vtu')
x,y,_=m.points.T

ms=meshio.read('output/'+str(infile_idx).zfill(4)+'_strain000099.vtu')
strain=np.sqrt(np.sum(ms.point_data['Strain']**2,axis=1))

plt.tripcolor(x,y,ms.cells[0].data,strain,vmax=3.27)
print(np.min(m.point_data['phi'][:,-1]),np.min(strain)+ iqr(strain,rng=(0,99)))

infile_idx=32
m=meshio.read('output/'+str(infile_idx).zfill(4)+'_configuration000099.vtu')
x,y,_=m.points.T

ms=meshio.read('output/'+str(infile_idx).zfill(4)+'_strain000099.vtu')
strain=np.sqrt(np.sum(ms.point_data['Strain']**2,axis=1))

plt.tripcolor(x,y,ms.cells[0].data,strain,vmax=3.27)
print(np.min(m.point_data['phi'][:,-1]),np.min(strain)+ iqr(strain,rng=(0,99)))

-0.45629618541986433 2.816401599908509
-0.4595397848328301 3.2712114922122173
